## 4 Modeling

---

In this phase, I will perform the following tasks:

1. Compare Models
2. Train Model
3. Evalueate Model
4. Conclusion
5. Refrences

**Setup**

Lead the necessary libraries and output data file from the previous step.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import joblib
import json
import warnings

from prophet import Prophet
from statsmodels.tsa.statespace.sarimax import SARIMAX
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

warnings.filterwarnings('ignore', category=DeprecationWarning)

avocados_df = pd.read_csv("../data/processed/avocado_processed.csv")
avocados_df.head()

### 4.1 Compare Models

---

In this section, I will have to compare the performance of different models and choose the best one. I will be using models with multi feature support (multivaried). Furthermore, since I want to predict the price of avocados fir the entire next month at once, I will be using multi-step forecasting. As this dataset is a time series data, I will compare the following models:

- ARIMA (AutoRegressive Integrated Moving Average), SARIMA (Seasonal ARIMA) and SARIMAX (multivaried Seasonal ARIMA)
- XGBoost (Extreme Gradient Boosting)
- Prophet (Facebook's Time Series Forecasting Library)

**ARIMA, SARIMA and SARIMAX**

These methods are all used for understanding and predicting future values of a timeseries dataset [1]. They have been widely used as powerful tools to predict future values based on historical data, however, they require a very good understanding of the data and can be sensitive how it's hyperparameters have been tuned [2]. Since SARIMA is a statistical model, the data should be stationary. SARIMAX is the multivaried version of SARIAM (univaried), accepting multiple exogenous features. 

**XGBoost**

XGBoost is an open-source library that provides efficient gradiant boosting implementation (trend-based forcasting). However, it mainly specializes in working with tree ensembles and is not specifically designed for time series data [3]. Furthermore, XGBoost has proved to be highly effective in modeling financial data, while methods like SARIMA exels at sales forcasting [4]. As a result, I will not use this model in my comparison.

**Prophet**

Prophet is a forecasting tool developed by Facebook. It is designed for analyzing time series that display patterns on different time scales such as yearly, weekly and daily. It also has advanced features for handling missing data, holidays, and other special events [5]. This model is the latest tool that has shown an improved performance in terms of accuracy of prediction [6]. Although it might provide a good performance with minimum tuning and understanding of the data, it can be prone to poor failure in some domains [2]. Eventhough this dataset does not contain any missing values so that I can leverage its ability to handle missing values, I will use this model to see if it can outperform the other models. This model does not need the inpit data to be stationary as it handles trends and seasonality it self. Prophet also supports multiple features and is also considered a suitable candidate.

> Note: As the seasonal decomopsition in the 'Data Understaning' section suggested, the data seems to be seasonal (SARIMA and Prophet are more suitable) rather than trending (ARIMA and XGBoost). Along with the facts mentioned above, I will only compare SARIMA and Prophet.

### 4.2 Train Model

---

In this section, I will first train a SARIMA model and then a Prophet model, and then compare them together to select the best performing one.

#### SARIMAX

In [ ]:
SARIMAX_DF = avocados_df.copy()
SARIMAX_DF["date"] = pd.to_datetime(avocados_df["date"])

target = "average_price"
features = ["total_volume", "4046", "4225", "4770", "type_conventional", "type_organic"]

sarimax_config = {
    'target': target,
    'features': features,
    'train_test_split': 0.7,
    'order': (1, 1, 1),
    'seasonal_order': (1, 1, 1, 12)
}

train_size = int(len(SARIMAX_DF) * sarimax_config['train_test_split'])
train, test = SARIMAX_DF[:train_size], SARIMAX_DF[train_size:]

sarimax_model = SARIMAX(
    train[sarimax_config['target']], 
    exog=train[sarimax_config['features']],
    order=sarimax_config['order'], 
    seasonal_order=sarimax_config['seasonal_order'],
)
sarimax_results = sarimax_model.fit()

sarimax_forecast = sarimax_results.get_forecast(steps=len(test), exog=test[sarimax_config['features']])
sarimax_pred = sarimax_forecast.predicted_mean

sarimax_mse = mean_squared_error(test[sarimax_config['target']], sarimax_pred)
sarimax_rmse = np.sqrt(sarimax_mse)
sarimax_mae = mean_absolute_error(test[sarimax_config['target']], sarimax_pred)
sarimax_r2 = r2_score(test[sarimax_config['target']], sarimax_pred)

print("SARIMAX Model Performance Metrics:")
print(f"Mean Squared Error (MSE): {sarimax_mse:.4f}")
print(f"Root Mean Squared Error (RMSE): {sarimax_rmse:.4f}")
print(f"Mean Absolute Error (MAE): {sarimax_mae:.4f}")
print(f"R-squared Score (R2): {sarimax_r2:.4f}")
print(f"Model Accuracy: {(1 - sarimax_mae) * 100:.2f}%")

Visualize the results of SARIMAX trained model:

In [ ]:
plt.figure(figsize=(12, 6))
plt.plot(SARIMAX_DF.index, SARIMAX_DF[sarimax_config['target']], label="Actual Prices", color='blue', linestyle='dashed')
plt.plot(test.index, sarimax_pred, label="SARIMAX Predicted Prices", color='red')

plt.xlabel("Date")
plt.ylabel("Average Price")
plt.title("Avocado Price Forecast using SARIMAX (Multiple Features)")
plt.legend()
plt.grid(True)

plt.show()

#### Prophet

In [ ]:
target = "average_price"
features = ["total_volume", "4046", "4225", "4770", "type_conventional", "type_organic"]

prophet_config = {
    'target': target,
    'features': features,
    'train_test_split': 0.8,
    'model_params': {
        'seasonality_mode': "multiplicative",
        'changepoint_prior_scale': 0.1,
        'weekly_seasonality': False,
        'yearly_seasonality': False,
        'daily_seasonality': False,
        'monthly_seasonality': {'name': 'monthly', 'period': 1, 'fourier_order': 5}
    }
}

PROPHET_DF = avocados_df.reset_index()[["date", prophet_config['target']] + prophet_config['features']].rename(
    columns={"date": "ds", prophet_config['target']: "y"}
)
PROPHET_DF["ds"] = pd.to_datetime(PROPHET_DF["ds"])
PROPHET_DF = PROPHET_DF.dropna()

train_size = int(len(PROPHET_DF) * prophet_config['train_test_split'])
train = PROPHET_DF[:train_size].copy()
test = PROPHET_DF[train_size:].copy()

prophet_model = Prophet(
    seasonality_mode=prophet_config['model_params']['seasonality_mode'],
    changepoint_prior_scale=prophet_config['model_params']['changepoint_prior_scale'],
    weekly_seasonality=prophet_config['model_params']['weekly_seasonality'],
    yearly_seasonality=prophet_config['model_params']['yearly_seasonality'],
    daily_seasonality=prophet_config['model_params']['daily_seasonality']
)

prophet_model.add_seasonality(**prophet_config['model_params']['monthly_seasonality'])

for feature in prophet_config['features']:
    prophet_model.add_regressor(feature)

prophet_model.fit(train)

future_dates = test[["ds"] + prophet_config['features']].copy()

prophet_forecast = prophet_model.predict(future_dates)

test = test.set_index("ds")
prophet_pred = prophet_forecast.set_index("ds")["yhat"].reindex(test.index)

prophet_mse = mean_squared_error(test["y"], prophet_pred)
prophet_rmse = np.sqrt(prophet_mse)
prophet_mae = mean_absolute_error(test["y"], prophet_pred)
prophet_r2 = r2_score(test["y"], prophet_pred)

print("\nOptimized Prophet Model Performance Metrics (Quarterly Seasonality):")
print(f"Mean Squared Error (MSE): {prophet_mse:.4f}")
print(f"Root Mean Squared Error (RMSE): {prophet_rmse:.4f}")
print(f"Mean Absolute Error (MAE): {prophet_mae:.4f}")
print(f"R-squared Score (R2): {prophet_r2:.4f}")
print(f"Model Accuracy: {(1 - prophet_mae) * 100:.2f}%")

Visualize the results of Prophet trained model:

In [ ]:
plt.figure(figsize=(12, 6))
plt.plot(PROPHET_DF["ds"], PROPHET_DF["y"], label="Actual Prices", color='blue', linestyle='dashed')
plt.plot(test.index, prophet_pred, label="Prophet Predicted Prices", color='green')

plt.xlabel("Date")
plt.ylabel("Average Price")
plt.title("Avocado Price Forecast using Prophet (Multiple Features)")
plt.legend()
plt.grid(True)

plt.show()

After some iterations, I realized that the Prophet model benefits from more training data. Hence, I changed the `train_test_split` from 0.7 to 0.8 (even though the data mining success criteria limits the size of the training set to 0.7).

By changing the `changepoint_prior_scale` from 0.05 to higher and lower values, the performance of the model did not show any improvement. Hence, I left it as 0.05.

> Note: It seems that the SARIMAX model is complaining about the data being non-stationary, even though I have already performed the stationarity test and the data is stationary. My data prepration step might have introduced this to the data. However, due to the time constraint, I will not be able to fix this issue.

### 4.3 Evalueate Models

---

For evaluating the accuracy of each model, I have used the following metrics:

- Mean Absolute Error (MAE) [Measures average of absolute errors.] [lower is better]
- Mean Squared Error (MSE) [Measures the average of squared errors.] [lower is better]
- R-squared (R²) [Measures the proportion of variance explained by model.] [higher is better]
- Mean Absolute Percentage Error (MAPE) [Measures error as a percentage of actual values.] [lower is better]

Each of these metrics is a measure of the difference between the predicted and actual values. Using multiple metrics helps with a more comprehensive evaluation of the model's performance. As shown below, the SARIMAX model has shown far better accuracy than the Prophet model. 

In [ ]:
model_metrics = {
    'Prophet': {
        'MAE (lower is better)': prophet_mae,
        'MSE (lower is better)': prophet_mse,
        'R² (higher is better)': prophet_r2,
        'RMSE (lower is better)': prophet_rmse
    },
    'SARIMAX': {
        'MAE (lower is better)': sarimax_mae,
        'MSE (lower is better)': sarimax_mse,
        'R² (higher is better)': sarimax_r2,
        'RMSE (lower is better)': sarimax_rmse
    }
}

fig, axes = plt.subplots(2, 2, figsize=(15, 12))
fig.suptitle('Model Performance Comparison', fontsize=16, y=1.02)

axes = axes.flatten()

colors = ['#2ecc71', '#3498db']

for idx, metric in enumerate(['MAE (lower is better)', 'MSE (lower is better)', 'R² (higher is better)', 'RMSE (lower is better)']):
    metric_values = [metrics[metric] for metrics in model_metrics.values()]
    
    bars = axes[idx].bar(model_metrics.keys(), metric_values, color=colors)
    
    for bar in bars:
        height = bar.get_height()
        axes[idx].text(bar.get_x() + bar.get_width()/2., height,
                      f'{height:.4f}',
                      ha='center', va='bottom')
    
    axes[idx].set_title(f'{metric}')
    axes[idx].grid(True, alpha=0.3)
    axes[idx].set_ylabel('Score')

plt.tight_layout()
plt.show()


It is evident that the SARIMAX model has shown far better accuracy than the Prophet model, even though SARIMAX is complaining about the data being non-stationary. This could be due to the small training set size, or misconfiguration of the model.

**Save Model**

Save the models and their configurations for future use.

In [21]:
joblib.dump(prophet_model, "../models/prophet/prophet_avocado_model.pkl")
with open("../models/prophet/prophet_config.json", "w") as f:
    json.dump(prophet_config, f, indent=4)

joblib.dump(sarimax_model, "../models/sarimax/sarimax_avocado_model.pkl") 
with open("../models/sarimax/sarimax_config.json", "w") as f:
    json.dump(sarimax_config, f, indent=4)

### 2.4 Conclusion

---

So far, I have trained a SARIMAX model and a Prophet model using multiple features on the same dataset, and evaluated their accuracy. The SARIMAX model has shown far better accuracy than the Prophet model. This could be due to many reasons, such as the small training set size, or misconfiguration of the model. As a result, I will use the SARIMAX model for the final prediction.

### 2.5 Refrences

----

[1] “Autoregressive integrated moving average,” Wikipedia, https://en.wikipedia.org/wiki/Autoregressive_integrated_moving_average (accessed Mar. 9, 2025).

[2] K. Kutzkov, “ARIMA vs Prophet vs LSTM for Time Series Prediction,” Neptune.ai, Jan. 24, 2025. [Online]. Available: https://neptune.ai/blog/arima-vs-prophet-vs-lstm. [Accessed: Mar. 9, 2025].

[3] “XGBoost,” Wikipedia, https://en.wikipedia.org/wiki/XGBoost (accessed Mar. 9, 2025).

[4] D. S. Chandel, R. Singh, B. Tiwari, A. Arora, and U. Mhatre, “A Journey into the Future: An Applied Exploration of Time Series Forecasting,” Databricks Community Technical Blog, Jan. 9, 2024. [Online]. Available: https://community.databricks.com/t5/technical-blog/a-journey-into-the-future-an-applied-exploration-of-time-series/ba-p/55494. [Accessed: Mar. 9, 2025].

[5] https://facebook.github.io/prophet/

[6] B. K. Jha and S. Pande, “Time Series Forecasting Model for Supermarket Sales using FB Prophet,” 2021 5th International Conference on Computing Methodologies and Communication (ICCMC), Erode, India, 2021, pp. 547-552.